<a href="https://colab.research.google.com/github/anhatsingh/hugging-face-learning/blob/main/GA_wk2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets tokenizers transformers &> /dev/null

In [ ]:
from datasets import load_dataset

ds = load_dataset("bookcorpus", split="all", trust_remote_code=True)
ds = ds.select(range(0, len(ds), 7))
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


bookcorpus.py:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/74004228 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 10572033
})

In [74]:
def get_batch(batch_size=1000):
  for i in range(0, len(ds), batch_size):
    yield ds[i: i+batch_size]['text']

In [75]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

In [76]:
text = "SEBI study finds 93% of individual F&O traders made losses between FY22 and FY24."

In [77]:
def tokenize_with_vocab_size(vocab_size = 5000):
  tokenizer = Tokenizer(BPE(unk_token='[UNK]'))
  tokenizer.normalizer = Lowercase()
  tokenizer.pre_tokenizer = Whitespace()

  trainer = BpeTrainer(
      vocab_size = vocab_size,
      special_tokens = ['[GO]', '[UNK]', '[PAD]', '[EOS]'],
      continuing_subword_prefix = '##',
      show_progress = True
  )

  tokenizer.train_from_iterator(
        iterator = get_batch(batch_size=10000),
        trainer = trainer,
        length = len(ds)
      )

  return tokenizer

In [78]:
tokenizer_5k = tokenize_with_vocab_size(5000)
tok = tokenizer_5k.encode(text).tokens
print(len(tok))

32


In [79]:
tokenizer_10k = tokenize_with_vocab_size(10000)
tok = tokenizer_10k.encode(text).tokens
print(len(tok))

28


In [80]:
tokenizer_15k = tokenize_with_vocab_size(15000)
tok = tokenizer_15k.encode(text).tokens
print(len(tok))

28


In [81]:
tokenizer_32k = tokenize_with_vocab_size(32000)
tok = tokenizer_32k.encode(text).tokens
print(len(tok))

25


In [82]:
!gdown 1QNnyh8iMN-IqW_h1w8gAMtw09Em7-e1e

Downloading...
From: https://drive.google.com/uc?id=1QNnyh8iMN-IqW_h1w8gAMtw09Em7-e1e
To: /content/hopper.json
100% 1.36M/1.36M [00:00<00:00, 179MB/s]


In [83]:
tokenizer_hopper = Tokenizer(BPE()).from_file("/content/hopper.json")

tok = tokenizer_hopper.encode(text).tokens
print(len(tok))

25


In [84]:
from pprint import pprint
print(tokenizer_32k.encode(text).tokens)
print(tokenizer_hopper.encode(text).tokens)

['seb', '##i', 'study', 'finds', '9', '##3', '%', 'of', 'individual', 'f', '&', 'o', 'traders', 'made', 'losses', 'between', 'f', '##y', '##22', 'and', 'f', '##y', '##2', '##4', '.']
['seb', '##i', 'study', 'finds', '9', '##3', '%', 'of', 'individual', 'f', '&', 'o', 'traders', 'made', 'losses', 'between', 'f', '##y', '##22', 'and', 'f', '##y', '##2', '##4', '.']


In [85]:
tokenizer_hopper.add_tokens(["FY"])

tok = tokenizer_hopper.encode(text).tokens
print(len(tok), tok)

22 ['seb', '##i', 'study', 'finds', '9', '##3', '%', 'of', 'individual', 'f', '&', 'o', 'traders', 'made', 'losses', 'between', 'fy', '22', 'and', 'fy', '24', '.']


In [86]:
from transformers import AutoTokenizer

bert = AutoTokenizer.from_pretrained("bert-base-uncased")
gpt2 = AutoTokenizer.from_pretrained("gpt2")

print(bert.all_special_tokens)
print(gpt2.all_special_tokens)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
['<|endoftext|>']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [87]:
imdb = load_dataset('stanfordnlp/imdb', split='train+test')
imdb

Dataset({
    features: ['text', 'label'],
    num_rows: 50000
})

In [88]:
def count_tokens(model: Tokenizer) -> int:
  s = 0
  encodings = model.encode_batch(imdb['text'])
  for i in range(len(encodings)):
    s += len(encodings[i])

  return s

def count_tokens2(model: AutoTokenizer) -> int:
  encodings = model.__call__(imdb['text'], return_length=True)
  s = sum(encodings.length)
  return s

x = [
    ("1", count_tokens(tokenizer_32k)),
    ("2", count_tokens2(bert)),
    ("3", count_tokens2(gpt2)),
    ("4", count_tokens(tokenizer_hopper))
]

print(sorted(x, key=lambda a: a[1]))

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


[('3', 14812432), ('1', 15352840), ('2', 15516058), ('4', 25600000)]
